In [1]:
# Import Libraries
import sqlite3
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import pairwise_distances
#!pip install tqdm
#from tqdm import tqdm

In [38]:
# Create database
db = sqlite3.connect("transactions.db")

# Connect to database
con = sqlite3.connect("/Users/DJ/E-Commerce-Recommendations/transactions.db")

In [39]:
# Load txt files into pandas and separate columns
adhesiveSealantTrans = pd.read_csv("/Users/DJ/E-Commerce-Recommendations/e-comm-data/Transactions with A&S.txt",sep='\t', header=0)
allTransaction = pd.read_csv("/Users/DJ/E-Commerce-Recommendations/e-comm-data/All Transations - 2 Weeks.txt", sep='\t', header=0)

In [40]:
# Check dataframes
adhesiveSealantTrans.head(5)

,order_number,l1,l2,l3,sku,brand
0,1182478,"Adhesives, Sealants and Tape",Putties,Putties,122821,2180
1,63908,"Adhesives, Sealants and Tape",Caulks,Caulks and Sealants,276172,1054
2,1165840,"Adhesives, Sealants and Tape",Concrete and Asphalt,Concrete Cleaners and Sealers,110119,4326
3,928748,Material Handling,Casters and Wheels,Plate Casters,71052,1793
4,1088891,"Adhesives, Sealants and Tape",Glues and Cements,Retaining Compounds,251285,2515


In [41]:
allTransaction.head(5)

,order_number,l1,l2,l3,sku,brand
0,168266,Power Tools,Power Saws and Accessories,Reciprocating Saw Blades,265105,2768
1,123986,Safety,Spill Control Supplies,Temporary Leak Repair,215839,586
2,158978,Hardware,Door Hardware,Thresholds,284756,1793
3,449035,"Electronics, Appliances, and Batteries",Batteries,Standard Batteries,12579,1231
4,781232,Motors,General Purpose AC Motors,General Purpose AC Motors,194681,2603


## Exploratory  Analysis

### SQL Queries

In [42]:
# Move dataframes to sql tables in transactions.db
adhesiveSealantTrans.to_sql("aNs_transactions", con, if_exists="replace", index=False)
allTransaction.to_sql("all_trans", con, if_exists="replace", index=False)

In [43]:
# The case study states the data was "sanitized", but let's make sure
# Check aNs_transactions for missing data
print("Adhesive & Sealant Missing Data")
print("Brand", pd.read_sql_query("SELECT CAST(SUM(CASE WHEN brand is NULL THEN 1 ELSE 0 END) as float)/COUNT(*) as ProportionMissing FROM ans_transactions;", con))
print("Sku", pd.read_sql_query("SELECT CAST(SUM(CASE WHEN sku is NULL THEN 1 ELSE 0 END) as float)/COUNT(*) as ProportionMissing FROM ans_transactions;", con))
print("L1", pd.read_sql_query("SELECT CAST(SUM(CASE WHEN l1 is NULL THEN 1 ELSE 0 END) as float)/COUNT(*) as ProportionMissing FROM ans_transactions;", con))
print("L2", pd.read_sql_query("SELECT CAST(SUM(CASE WHEN l2 is NULL THEN 1 ELSE 0 END) as float)/COUNT(*) as ProportionMissing FROM ans_transactions;", con))
print("L3", pd.read_sql_query("SELECT CAST(SUM(CASE WHEN l3 is NULL THEN 1 ELSE 0 END) as float)/COUNT(*) as ProportionMissing FROM ans_transactions;", con))

# Check all_trans for missing data
print("\nAll Transactions Missing Data")
print("Brand", pd.read_sql_query("SELECT CAST(SUM(CASE WHEN brand is NULL THEN 1 ELSE 0 END) as float)/COUNT(*) as ProportionMissing FROM all_trans;", con))
print("Sku", pd.read_sql_query("SELECT CAST(SUM(CASE WHEN sku is NULL THEN 1 ELSE 0 END) as float)/COUNT(*) as ProportionMissing FROM all_trans;", con))
print("L1", pd.read_sql_query("SELECT CAST(SUM(CASE WHEN l1 is NULL THEN 1 ELSE 0 END) as float)/COUNT(*) as ProportionMissing FROM all_trans;", con))
print("L2", pd.read_sql_query("SELECT CAST(SUM(CASE WHEN l2 is NULL THEN 1 ELSE 0 END) as float)/COUNT(*) as ProportionMissing FROM all_trans;", con))
print("L3", pd.read_sql_query("SELECT CAST(SUM(CASE WHEN l3 is NULL THEN 1 ELSE 0 END) as float)/COUNT(*) as ProportionMissing FROM all_trans;", con))


Adhesive & Sealant Missing Data
Brand    ProportionMissing
0                0.0
Sku    ProportionMissing
0                0.0
L1    ProportionMissing
0                0.0
L2    ProportionMissing
0                0.0
L3    ProportionMissing
0                0.0

All Transactions Missing Data
Brand    ProportionMissing
0                0.0
Sku    ProportionMissing
0                0.0
L1    ProportionMissing
0                0.0
L2    ProportionMissing
0                0.0
L3    ProportionMissing
0                0.0


In [44]:
# See what items sell the most with same transaction as adhesives and sealants, top 30 to find 3 different complement categories
pd.read_sql_query("SELECT l1,l2, l3, COUNT(sku) FROM ans_transactions GROUP BY sku ORDER BY COUNT(sku) DESC LIMIT 30;", con)

# Batteries are the most sold sku that are not in adhesives and sealants category, followed by hand wipes, gloves, and 
# paper towels. This makes batteries, cleaning supplies, and safety items the most purchased complements.

,l1,l2,l3,COUNT(sku)
0,"Adhesives, Sealants and Tape",Tapes,Electrical Tapes,33186
1,"Electronics, Appliances, and Batteries",Batteries,Standard Batteries,20845
2,"Electronics, Appliances, and Batteries",Batteries,Standard Batteries,18142
3,"Adhesives, Sealants and Tape",Thread and Gasket Sealants,Pipe Sealant Tape,10161
4,"Adhesives, Sealants and Tape",Tapes,Duct and Cloth Tapes,9364
5,"Adhesives, Sealants and Tape",Tapes,Duct and Cloth Tapes,7910
6,"Adhesives, Sealants and Tape",Thread and Gasket Sealants,Pipe Sealant Tape,7609
7,"Adhesives, Sealants and Tape",Tapes,Electrical Tapes,7060
8,"Electronics, Appliances, and Batteries",Batteries,Standard Batteries,6771
9,"Adhesives, Sealants and Tape",Tapes,Electrical Tapes,6494


In [45]:
# See what are top 10 items in ALL transactons
pd.read_sql_query("SELECT l1,l2, l3, COUNT(sku) FROM all_trans GROUP BY sku ORDER BY COUNT(sku) DESC LIMIT 10;", con)

# Batteries are most sold item, followed by paper cleaning products and air filters

,l1,l2,l3,COUNT(sku)
0,"Electronics, Appliances, and Batteries",Batteries,Standard Batteries,9234
1,"Electronics, Appliances, and Batteries",Batteries,Standard Batteries,7408
2,Safety,Footwear and Footwear Accessories,Socks,5721
3,Cleaning,Paper Products and Dispensers,"Paper Towels, Rolls",4965
4,"Electronics, Appliances, and Batteries",Batteries,Standard Batteries,4210
5,HVAC and Refrigeration,Air Filters,Pleated Air Filters,3261
6,HVAC and Refrigeration,Air Filters,Pleated Air Filters,3169
7,Cleaning,Paper Products and Dispensers,Toilet Paper,2955
8,HVAC and Refrigeration,Air Filters,Pleated Air Filters,2933
9,"Electronics, Appliances, and Batteries",Batteries,Standard Batteries,2831


### Pandas Exploration

In [46]:
# Check data types
print(" Adhesive & Sealant Data\n", adhesiveSealantTrans.dtypes)
print("\n All Transactions Data\n", allTransaction.dtypes)

 Adhesive & Sealant Data
 order_number     int64
l1              object
l2              object
l3              object
sku              int64
brand            int64
dtype: object

 All Transactions Data
 order_number     int64
l1              object
l2              object
l3              object
sku              int64
brand            int64
dtype: object


In [47]:
# See unique values for all features
print(" Adhesive & Sealant Data\n", adhesiveSealantTrans.nunique())
print("\n All Transactions Data\n", allTransaction.nunique())

 Adhesive & Sealant Data
 order_number    479917
l1                  33
l2                 586
l3                5771
sku             224423
brand             4296
dtype: int64

 All Transactions Data
 order_number    801575
l1                  33
l2                 593
l3                6203
sku             275958
brand             4574
dtype: int64


In [48]:
# Which Sku's are most popular?
aNs_skuCount = pd.DataFrame(adhesiveSealantTrans.groupby('sku')['order_number'].count())
print("Adhesives & Sealants\n", aNs_skuCount.sort_values('order_number', ascending=False).head())

all_skuCount = pd.DataFrame(allTransaction.groupby('sku')['order_number'].count())
print("\nAll Transactions\n", all_skuCount.sort_values('order_number', ascending=False).head())

Adhesives & Sealants
         order_number
sku                 
100164         33186
282471         20845
282473         18142
265850         10161
309746          9364

All Transactions
         order_number
sku                 
282471          9234
282473          7408
154210          5721
184171          4965
282469          4210


In [49]:
# Which items are these?
aNsItems = adhesiveSealantTrans[['sku', 'l3']]
print("Adhesives & Sealants\n", aNsItems.head())

allItems = allTransaction[['sku', 'l3']]
print("\nAll Transactions\n", allItems.head())

print("\nTop 5 Adhesive & sealant Items")
print(aNsItems[aNsItems['sku']==100164].head(1))
print(aNsItems[aNsItems['sku']==282471].head(1))
print(aNsItems[aNsItems['sku']==282473].head(1))
print(aNsItems[aNsItems['sku']==265850].head(1))
print(aNsItems[aNsItems['sku']==309746].head(1))

print("\nTop 5 All Items")
print(allItems[allItems['sku']==282471].head(1))
print(allItems[allItems['sku']==282473].head(1))
print(allItems[allItems['sku']==154210].head(1))
print(allItems[allItems['sku']==184171].head(1))
print(allItems[allItems['sku']==282469].head(1))

#Batteries are most popular sku in both dataframes

Adhesives & Sealants
       sku                             l3
0  122821                        Putties
1  276172            Caulks and Sealants
2  110119  Concrete Cleaners and Sealers
3   71052                  Plate Casters
4  251285            Retaining Compounds

All Transactions
       sku                         l3
0  265105   Reciprocating Saw Blades
1  215839      Temporary Leak Repair
2  284756                 Thresholds
3   12579         Standard Batteries
4  194681  General Purpose AC Motors

Top 5 Adhesive & sealant Items
        sku                l3
100  100164  Electrical Tapes
       sku                  l3
84  282471  Standard Batteries
        sku                  l3
129  282473  Standard Batteries
        sku                 l3
485  265850  Pipe Sealant Tape
        sku                    l3
492  309746  Duct and Cloth Tapes

Top 5 All Items
        sku                  l3
473  282471  Standard Batteries
        sku                  l3
389  282473  Standard Batterie

## Building a recommendation system

Given the limited amount of data available, we will use collaborative filtering using the items in each order number to score the similarity of each item based on which ones are purchased together and generate recommendations based on the similarity score.

The recommendation system will be created with allTransactions dataframe and then tested with sku's from adhesiveSealantTrans dataframe to see what is recommended.

### Prepare Data

Group by order number and drop less than 10    

In [50]:
# Check shape before dropping rows
allTransaction.shape

(2107537, 6)

In [51]:
# Group orders
grouped = allTransaction.groupby('order_number')

In [52]:
# Filter out orders that have less than 10 items
allTransaction = grouped.filter(lambda x: len(x)>10)

In [53]:
# Check shape after dropping rows
allTransaction.shape

(499972, 6)

In [18]:
# allTransaction['sku'].head()

In [54]:
# Drop most broad categories and keep most granular
allTransaction.drop('l1', axis=1, inplace=True)
allTransaction.drop('l2', axis=1, inplace=True)

/Users/DJ/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [55]:
# one-hot encode columns
allTransaction = pd.get_dummies(allTransaction)
allTransaction.head()

,order_number,sku,brand,l3_12 Volt Accessories,l3_12-Point Flange Head Cap Screws,l3_3-Ring Binder Accessories,l3_3-Ring Binders,l3_3.3 Inch Diameter Motors,l3_4.4 Inch Diameter Motors,l3_5 X 20mm Glass and Ceramic Fuses,...,l3_Worm Gear Clamps,l3_Wrap-a-Round Tape Measures,l3_Wrist Rests and Palm Supports,l3_Wrist Supports and Wraps,l3_Y Strainers,l3_Yard Hydrants,l3_Yoke Ends,l3_Zone Valve Actuators,l3_Zone Valves,l3_pH Meters
22,173984,46841,1726,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
37,649979,299689,2163,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
58,482732,38789,4477,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
64,173984,264751,3712,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
66,339808,292307,4572,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [56]:
# Shape after 
allTransaction.shape

(499972, 4463)

In [57]:
# Drop sku and brand because they do not have enough info to be useful
allTransaction.drop('sku', axis=1, inplace=True)
allTransaction.drop('brand', axis=1, inplace=True)

In [58]:
grouped = allTransaction.groupby('order_number')


In [59]:
# Consolidate data by joining order numbers and summing other data 
data_cols = list(allTransaction.columns)
data_cols.remove('order_number')
allTransaction = allTransaction.groupby('order_number').sum()[data_cols].reset_index()

In [60]:
print(allTransaction.shape)
allTransaction.head()

(25720, 4461)


,order_number,l3_12 Volt Accessories,l3_12-Point Flange Head Cap Screws,l3_3-Ring Binder Accessories,l3_3-Ring Binders,l3_3.3 Inch Diameter Motors,l3_4.4 Inch Diameter Motors,l3_5 X 20mm Glass and Ceramic Fuses,l3_5S Red Tag Stations,l3_A/C Conduit Kits,...,l3_Worm Gear Clamps,l3_Wrap-a-Round Tape Measures,l3_Wrist Rests and Palm Supports,l3_Wrist Supports and Wraps,l3_Y Strainers,l3_Yard Hydrants,l3_Yoke Ends,l3_Zone Valve Actuators,l3_Zone Valves,l3_pH Meters
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,66284,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,66334,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,66344,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,66361,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [62]:
# Drop order numbers so they are not included in recommendations
allTransaction = allTransaction.set_index('order_number')
allTransaction.head()


,l3_12 Volt Accessories,l3_12-Point Flange Head Cap Screws,l3_3-Ring Binder Accessories,l3_3-Ring Binders,l3_3.3 Inch Diameter Motors,l3_4.4 Inch Diameter Motors,l3_5 X 20mm Glass and Ceramic Fuses,l3_5S Red Tag Stations,l3_A/C Conduit Kits,l3_A/C Mounting Pads,...,l3_Worm Gear Clamps,l3_Wrap-a-Round Tape Measures,l3_Wrist Rests and Palm Supports,l3_Wrist Supports and Wraps,l3_Y Strainers,l3_Yard Hydrants,l3_Yoke Ends,l3_Zone Valve Actuators,l3_Zone Valves,l3_pH Meters
order_number,,,,,,,,,,,,,,,,,,,,,
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
66284,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
66334,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
66344,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
66361,0,0,0,0,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0


In [ ]:
# Create list of items to recommend
# items = list(allTransaction.columns)
# print(items)

## Create recommendation engine

In [63]:
# Create a similarity matrix using hamming to calculate jaccard similarity

jac_similarity_matrix = 1 - pairwise_distances(allTransaction.T, metric = "hamming")
jac_similarity_matrix = pd.DataFrame(jac_similarity_matrix, index=allTransaction.columns, columns=allTransaction.columns)
jac_similarity_matrix

,l3_12 Volt Accessories,l3_12-Point Flange Head Cap Screws,l3_3-Ring Binder Accessories,l3_3-Ring Binders,l3_3.3 Inch Diameter Motors,l3_4.4 Inch Diameter Motors,l3_5 X 20mm Glass and Ceramic Fuses,l3_5S Red Tag Stations,l3_A/C Conduit Kits,l3_A/C Mounting Pads,...,l3_Worm Gear Clamps,l3_Wrap-a-Round Tape Measures,l3_Wrist Rests and Palm Supports,l3_Wrist Supports and Wraps,l3_Y Strainers,l3_Yard Hydrants,l3_Yoke Ends,l3_Zone Valve Actuators,l3_Zone Valves,l3_pH Meters
l3_12 Volt Accessories,1.000000,0.999650,0.999572,0.998639,0.998523,0.999378,0.994907,0.999650,0.999650,0.999611,...,0.950389,0.998289,0.999650,0.997045,0.998406,0.999495,0.999650,0.999650,0.999495,0.999300
l3_12-Point Flange Head Cap Screws,0.999650,1.000000,0.999844,0.998911,0.998717,0.999650,0.995179,0.999922,0.999922,0.999883,...,0.950700,0.998561,0.999922,0.997317,0.998678,0.999767,0.999922,0.999922,0.999767,0.999572
l3_3-Ring Binder Accessories,0.999572,0.999844,1.000000,0.998950,0.998639,0.999572,0.995101,0.999844,0.999844,0.999806,...,0.950622,0.998484,0.999844,0.997240,0.998600,0.999689,0.999844,0.999844,0.999689,0.999495
l3_3-Ring Binders,0.998639,0.998911,0.998950,1.000000,0.997706,0.998639,0.994168,0.998911,0.998911,0.998872,...,0.949806,0.997551,0.998911,0.996306,0.997667,0.998756,0.998911,0.998911,0.998756,0.998561
l3_3.3 Inch Diameter Motors,0.998523,0.998717,0.998639,0.997706,1.000000,0.998600,0.993974,0.998717,0.998717,0.998678,...,0.949495,0.997356,0.998717,0.996112,0.997473,0.998561,0.998717,0.998717,0.998561,0.998367
l3_4.4 Inch Diameter Motors,0.999378,0.999650,0.999572,0.998639,0.998600,1.000000,0.994907,0.999650,0.999650,0.999611,...,0.950389,0.998289,0.999650,0.997045,0.998406,0.999495,0.999650,0.999650,0.999495,0.999300
l3_5 X 20mm Glass and Ceramic Fuses,0.994907,0.995179,0.995101,0.994168,0.993974,0.994907,1.000000,0.995179,0.995179,0.995140,...,0.946812,0.993818,0.995179,0.992574,0.993935,0.995023,0.995179,0.995179,0.995023,0.994829
l3_5S Red Tag Stations,0.999650,0.999922,0.999844,0.998911,0.998717,0.999650,0.995179,1.000000,0.999922,0.999883,...,0.950661,0.998561,0.999922,0.997317,0.998678,0.999767,0.999922,0.999922,0.999767,0.999572
l3_A/C Conduit Kits,0.999650,0.999922,0.999844,0.998911,0.998717,0.999650,0.995179,0.999922,1.000000,0.999883,...,0.950661,0.998561,0.999922,0.997317,0.998678,0.999767,0.999922,0.999922,0.999767,0.999572
l3_A/C Mounting Pads,0.999611,0.999883,0.999806,0.998872,0.998678,0.999611,0.995140,0.999883,0.999883,1.000000,...,0.950700,0.998523,0.999883,0.997278,0.998639,0.999728,0.999883,0.999883,0.999728,0.999533


In [64]:
# Score "users" using dot porduct of 
user_score = allTransaction[allTransaction.columns].dot(jac_similarity_matrix)

In [66]:
user_score.head()

,l3_12 Volt Accessories,l3_12-Point Flange Head Cap Screws,l3_3-Ring Binder Accessories,l3_3-Ring Binders,l3_3.3 Inch Diameter Motors,l3_4.4 Inch Diameter Motors,l3_5 X 20mm Glass and Ceramic Fuses,l3_5S Red Tag Stations,l3_A/C Conduit Kits,l3_A/C Mounting Pads,...,l3_Worm Gear Clamps,l3_Wrap-a-Round Tape Measures,l3_Wrist Rests and Palm Supports,l3_Wrist Supports and Wraps,l3_Y Strainers,l3_Yard Hydrants,l3_Yoke Ends,l3_Zone Valve Actuators,l3_Zone Valves,l3_pH Meters
order_number,,,,,,,,,,,,,,,,,,,,,
2,14.811159,14.814736,14.813725,14.799728,14.796695,14.810653,14.744518,14.814736,14.814736,14.814152,...,14.078694,14.794518,14.814736,14.777644,14.796229,14.812403,14.814736,14.814736,14.812403,14.809642
66284,13.478110,13.481299,13.480404,13.467924,13.465047,13.477644,13.416913,13.481299,13.481454,13.481182,...,12.816680,13.465047,13.481376,13.446501,13.464269,13.479121,13.481299,13.481299,13.479199,13.476944
66334,62.318040,62.333942,62.329705,62.270956,62.258787,62.316446,62.036275,62.333942,62.334253,62.332387,...,59.268896,62.252488,62.333942,62.182076,62.254355,62.325039,62.333865,62.333865,62.324028,62.312014
66344,15.861236,15.864347,15.863180,15.848328,15.845918,15.860070,15.790280,15.864347,15.864347,15.863880,...,15.092030,15.842652,15.864580,15.822900,15.844751,15.862869,15.864347,15.864347,15.861858,15.858826
66361,210.445179,210.499028,210.485264,210.289697,210.247823,210.441641,209.500117,210.499028,210.499222,210.505871,...,200.234759,210.215708,210.499689,209.953344,210.236081,210.471034,210.499028,210.499028,210.466524,210.426361


In [73]:
# Generate recs
# user_col = allTransaction.order_number
# users = allTransaction.loc[:,allTransaction[user_col]]

In [ ]:
# Save

In [ ]:
# Print